In [1]:
%matplotlib qt
import os
root = os.path.realpath(os.getcwd()+ os.sep+"..")
import sys
sys.path.append(root)
from modules.WiFi import WiFi, Modes, make_command
import pathfinder, pathfinder_visualizer
import importlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim

In [21]:
from ipywidgets import Button, HBox, ToggleButton, TwoByTwoLayout, Layout
# 3 buttons. Start service, stop service, show visualization
make_layout = lambda: Layout(width="auto", height="auto")
start_button = Button(description="Start Pathfinder", disabled=pathfinder._service.is_running(), button_style='primary', layout=make_layout())
stop_button = Button(description="Stop Pathfinder", disabled=not pathfinder._service.is_running(), button_style='danger', layout=make_layout())
show_button = Button(description="Show Visualization", button_style='warning', layout=make_layout())

gui = TwoByTwoLayout(
    top_left=start_button,
    bottom_right=stop_button,
    # top_right=show_button
    )

called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service
called stop_path_finder_service

Called stop_worldview_service
Called stop_lidar_service

started Lidar Scanning Thread
waiting for obstacle thread to join
waiting for lidar thread to join
Stopping Lidar Service
Starting Worldview Service
Stopping Pathfinder Service
called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service

started Lidar Scanning Thread


Too many measurments in the input buffer: 561/500. Clearing buffer...
Too many measurments in the input buffer: 510/500. Clearing buffer...
Too many measurments in the input buffer: 560/500. Clearing buffer...
Too many measurments in the input buffer: 560/500. Clearing buffer...
Too many measurments in the input buffer: 540/500. Clearing buffer...
Too many measurments in the input buffer: 515/500. Clearing buffer...
Too many measurments in the input buffer: 559/500. Clearing buffer...
Too many measurments in the input buffer: 542/500. Clearing buffer...
Too many measurments in the input buffer: 527/500. Clearing buffer...
Too many measurments in the input buffer: 539/500. Clearing buffer...
Too many measurments in the input buffer: 558/500. Clearing buffer...
Too many measurments in the input buffer: 542/500. Clearing buffer...
Too many measurments in the input buffer: 515/500. Clearing buffer...
Too many measurments in the input buffer: 575/500. Clearing buffer...
Too many measurments

called stop_path_finder_service

Called stop_worldview_service
Called stop_lidar_service
waiting for obstacle thread to join
waiting for lidar thread to join
Stopping Lidar Service
Starting Worldview Service
Stopping Pathfinder Service


In [22]:
def start_pathfinder(*args):
    if pathfinder._service.is_running():
        print("Service is already running", file=sys.stderr)
        return
    pathfinder.start_pathfinder_service()
    stop_button.disabled=False
    start_button.disabled=True
start_button.on_click(start_pathfinder)
def stop_pathfinder(*args):
    pathfinder.stop_pathfinder_service()
    importlib.reload(pathfinder)
    stop_button.disabled=True
    start_button.disabled=False
stop_button.on_click(stop_pathfinder)
# def show_visual(*args):
#     fig, update_func = pathfinder_visualizer.run_visualizer(pathfinder)
#     anime = anim.FuncAnimation(fig, update_func, 1, interval=50, blit=True)
#     plt.show()
#     return anime
# show_button.on_click(show_visual)

In [23]:
gui

TwoByTwoLayout(children=(Button(button_style='primary', description='Start Pathfinder', layout=Layout(grid_are…

In [24]:
def func():
    fig, update_func = pathfinder_visualizer.run_visualizer(pathfinder)
    anime = anim.FuncAnimation(fig, update_func, 1, interval=50, blit=True)
    plt.show()
    return anime
func()